# 기계 번역이 걸어온 길

번역 모델이 발전해 온 과정을 살펴보고, 번역을 생성하는 여러 가지 방법을 소개합니다. 자연어 처리에서 DATA Augmentation을 어떻게 할 수 있는지, 자연어 처리 성능은 어떻게 측정할 수 있는지 알아봅니다. 기계 번역과 뗄 수 없는 챗봇도 간단히 공부합니다.


- 들어가며
- 번역의 흐름
- 지적 생성을 위한 넓고 얕은 탐색
  - greedy decoding
  - beam search
  - sampling
- 방과 후 번역 수업
  - data augmentation
  - lexical substitution
  - back translation
  - random noise injection
- metric
- 챗봇
  - 챗봇과 번역
  - 좋은 챗봇이란?
  - 대표 챗봇

# 들어가며

학습 목표

- 번역 모델 발전 과정
- 번역 생성 방법
- 더 높은 성능을 위해서
- 성능 측정 지표

# 번역의 흐름

Machine Translation == MT

|[규칙 기반 기계 번역](https://m.blog.naver.com/newheater/221763031420)|[통계적 기계 번역](https://wikidocs.net/21687)|
|:-:|:-:|
|Rule-Based MT|Statistical MT|
|규칙을 정함|예측 확률 계산|
|유연성 결여, 복잡성, 개발 시간||
|나는 -> I am|$$p(1|I) * p(1|1, 2) * p(나|I)$$|

[SMT 추가 설명](https://en.wikipedia.org/wiki/Statistical_machine_translation)

$$ p(번역 단어 갯수|원문 단어) * p(번역 단어 위치|원문 단어 위치, 번역문 길이) * p(번역 단어|원문 단어$$


# 지적 생성을 위한 넓고 얕은 탐색

## Greedy Decoding

![https://en.wikipedia.org/wiki/Greedy_algorithm](https://upload.wikimedia.org/wikipedia/commons/8/8c/Greedy-search-path-example.gif)

가장 높은 확률을 갖는 단어가 다음 단어로!

have -> 가지다?, 마시다?

대부분 가지다라고 사용됨

$$O(V^L)$$

## Bean Search

In [1]:
import math
import numpy as np

def beam_search_decoder(prob, beam_size):
  sequences = [[[], 1.0]]  # 생성된 문장과 점수를 저장

  for tok in prob:
    all_candidates = []

    for seq, score in sequences:
      for idx, p in enumerate(tok): # 각 단어의 확률을 총점에 누적 곱
        candidate = [seq + [idx], score * -math.log(-(p-1))]
        all_candidates.append(candidate)

    ordered = sorted(all_candidates,
                    key=lambda tup:tup[1],
                    reverse=True) # 총점 순 정렬
    sequences = ordered[:beam_size] # Beam Size에 해당하는 문장만 저장 

  return sequences

In [2]:
vocab = {
    0: "<pad>",
    1: "까요?",
    2: "커피",
    3: "마셔",
    4: "가져",
    5: "될",
    6: "를",
    7: "한",
    8: "잔",
    9: "도",
}

# row가 vocab
# col이 순간

prob_seq = [[0.01, 0.01, 0.60, 0.32, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01],
            [0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.75, 0.01, 0.01, 0.17],
            [0.01, 0.01, 0.01, 0.35, 0.48, 0.10, 0.01, 0.01, 0.01, 0.01],
            [0.24, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.68],
            [0.01, 0.01, 0.12, 0.01, 0.01, 0.80, 0.01, 0.01, 0.01, 0.01],
            [0.01, 0.81, 0.01, 0.01, 0.01, 0.01, 0.11, 0.01, 0.01, 0.01],
            [0.70, 0.22, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01],
            [0.91, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01],
            [0.91, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01],
            [0.91, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01]]

prob_seq = np.array(prob_seq)
beam_size = 3

result = beam_search_decoder(prob_seq, beam_size)

for seq, score in result:
  sentence = ""

  for word in seq:
    sentence += vocab[word] + " "

  print(sentence, "// Score: %.4f" % score)


커피 를 가져 도 될 까요? <pad> <pad> <pad> <pad>  // Score: 42.5243
커피 를 마셔 도 될 까요? <pad> <pad> <pad> <pad>  // Score: 28.0135
마셔 를 가져 도 될 까요? <pad> <pad> <pad> <pad>  // Score: 17.8983


![https://towardsdatascience.com/foundations-of-nlp-explained-visually-beam-search-how-it-works-1586b9849a24](https://miro.medium.com/max/1400/1*tEjhWqUgjX37VnT7gJN-4g.png)

모델 학습 단계에서 Beam Search를 사용하지는 않음

## [Sampling](https://towardsdatascience.com/decoding-strategies-that-you-need-to-know-for-response-generation-ba95ee0faadc)

![https://towardsdatascience.com/decoding-strategies-that-you-need-to-know-for-response-generation-ba95ee0faadc](https://miro.medium.com/max/1400/1*qC-KFHQLToPASxawmUPWLQ.png)

확률적으로 다양한 단어를 뽑음

확률 분포를 따라가기 때문에, 뜬금없는 문장 생성을 피함

서비스 불가

# [Data Augmentation](https://nittaku.tistory.com/272)

이미지는 직관적인 다양한 방법이 존재함

- 좌우 반전 (Flipping과 중복): 이미지를 좌우로 반전시킨다.
- Crop: 이미지의 특정 부분을 잘라낸다.
- 밝기 조절: 이미지의 밝기를 변화시킨다.
- Rotation: 이미지를 0 ~ 360도 회전시킨다.
- Shifting: 이미지를 약 10px 움직인다. 끝단이 잘려 나가는 효과가 있다.
- Rescaling: 이미지를 1.0 ~ 1.6배 키운다.
- Flipping: 이미지를 상하 / 좌우 반전을 한다.
- Shearing: 약 -20 ~ 20도 정도로 이미지를 찌그러트린다.
- Stretching: 이미지를 가로 / 세로 약 1.0 ~ 1.3배 늘린다.

## Lexical Substitution

어휘 대체

### 동의어 기반 대체

시소러스(Thesaurus)란, 어떤 단어의 동의어나 유의어를 집중적으로 구축해놓은 사전!

[워드넷](https://kh-kim.gitbook.io/natural-language-processing-with-pytorch/00-cover-4/03-wordnet)

[speech and language processing](https://web.stanford.edu/~jurafsky/slp3/ed3book.pdf)

[korlex](http://korlex.pusan.ac.kr/search/WebApplication2/KorLex_SearchPage.aspx)

모든 것을 사람이 구축해야함

### Embedding 활용 대체

Pre-training Word Embedding을 사용하여, 단어를 대체함

![https://medium.com/@hari4om/word-embedding-d816f643140](https://miro.medium.com/max/1400/1*sAJdxEsDjsPMioHyzlN3_A.png)

### TF-IDF 기반 대체

TF-IDF에서 중요도가 높은 내용을 변경하지 않고,낮은 내용을 선별적으로 변경함

## [Back Translation](https://dev-sngwn.github.io/2020-01-07-back-translation/)

![](https://raw.githubusercontent.com/dev-sngwn/dev-sngwn.github.io/master/_posts/assets/2020-01-07-back-translation/03_flu.png)

유창하게 만들자!

target에만 의존하게 되겠다?

![](https://raw.githubusercontent.com/dev-sngwn/dev-sngwn.github.io/master/_posts/assets/2020-01-07-back-translation/04_synthetic.png)

학습 결과를 토대로 source를 만들자! == Back Trranslation

SOTA(state-of-the-art)

## Random Noise Injection

### 오타 노이즈 추가

![https://zeddios.tistory.com/640](https://t1.daumcdn.net/cfile/tistory/998B68345C0E05C230)

ㅋㅋㅋㅋㅋㅋㅌㅌㅋㅋㅋㅋㅌ

키보드 상 거리를 기반으로 노이즈 추가

### 공백 노이즈 추가

Placeholder Token(이하 공백 토큰)

나는 고양이를 종아한다

나는 _ 종아한다

학습의 과적합을 방지

### 랜덤 유의어 추가

유사 단어를 골라 아무렇게 삽입



# 채점은 어떻게?

BLEU(Bilingual Evaluation Understudy) Score

$$BLEU = min(1, \frac{output\ length(예측\ 문장)}{reference\ length(실제\ 문장)})(\prod^4_{i=1}precision_i)^{\frac{1}{4}}$$


'기계가 실제 번역을 얼마나 잘 재현했는가?' 를 평가하는 지표

[장단점](https://wikidocs.net/31695)

같은 단어 반복 방어, 어순 중요, 짧은 번역 X

[GLUE](https://huffon.github.io/2019/11/16/glue/)

Fine-Tuning 모델의 경우 좋은 결과물이 나옴

# 챗봇!

## [챗봇과 번역기](https://norux.me/56)

검색 기반 vs 생성 기반

Open Domain vs Closed Domain

긴 대화 vs 짧은 대화

질문에 대한 답변은 결국 수렴함

## 좋은 챗봇

### 200ms

답변의 공백 마지노선

### 시공간을 담은 질문

오늘(시간) 대구(공간) 날씨 어때?

### [페르소나](https://doc.danbee.ai/blog_005_UX04_persona.html)

제품이나 서비스 목적에 맞게 설정해야함

인격의 일관성(Coherent Personality)

### [대화의 일관성](https://arxiv.org/pdf/1606.01541.pdf)

쉬운 답변, 새로운 정보, 대화의 일관성을 점수로 강화학습을 진행함



## 대표적인 챗봇

### [Meena](https://brunch.co.kr/@synabreu/35)

### [Blender](https://littlefoxdiary.tistory.com/39)